In [1]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 32.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 5.2 MB/s 
     |████████████████████████████████| 69 kB 9.2 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=faa290903f0afd3fb9e7e86a71e326ac93eeb63ca58f2585728a81898ddca99a
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1.9-py2.py3-none-any.whl size=64666 sha256=f8abd0c9822ea5081751209cf5a1e53f76da6fe5d0a708d1c2e0752ae4168d6b
  Stored in directory: /root/.cache/pip/wheels/11/d8/85/01a8cf50977badf2da880865124b159b112b861655a9b631d3
Successfully built pycuda pytools


Standard imports.


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

PyCUDA imports

In [3]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

```iDivUp``` function: if ```b``` divides ```a```, then ```a/b``` is returned, otherwise the function returns the integer division between ```a``` and ```b``` summed to 1```.

In [4]:
###################
# iDivUp FUNCTION #
###################
def iDivUp(a, b):
    # Round a / b to nearest higher integer value
    a = np.int32(a)
    b = np.int32(b)
    return (a / b + 1) if (a % b != 0) else (a / b)

Kernel functions.

In [5]:
BLOCKSIZE = 256

mod = SourceModule("""

#define BLOCKSIZE		%(BBLOCKSIZE)s

#define PI				3.1415926535897932384626433832795028841971693993751058209749445923078164062

/********************************/
/* PROPAGATION DEVICE FUNCTIONS */
/********************************/
template<class T>
__host__ __device__  T propagatingFunctionStationary(const T x, const T x1, const T x2) { return cos((3 * PI / (x2 - x1)) * x); }

template<class T>
__host__ __device__  T propagatingFunctionStationaryDerivative(const T x, const T x1, const T x2) { return -(3 * PI / (x2 - x1)) * sin((3 * PI / (x2 - x1)) * x); }

/*************************/
/* STEP0 DEVICE FUNCTION */
/*************************/
template<class T>
__device__ void setStep0Device(T * __restrict__ d_u1, T * __restrict__ d_u, const T * __restrict__ d_t, const T * __restrict__ d_x, const T v, const T x1, const T x2, const int tid) {
  
  d_u1[tid] = ((T)0.5) * (propagatingFunctionStationary(d_x[tid] - x1 - v * d_t[0], x1, x2) - 
					        propagatingFunctionStationary(d_x[tid] - x1 + v * d_t[0], x1, x2));   
	d_u[tid] = d_u1[tid]; }

/*************************/
/* STEP1 DEVICE FUNCTION */
/*************************/
template<class T>
__device__ void setStep1Device(T * __restrict__ d_u1, T * __restrict__ d_u2, T * __restrict__ d_u, const T * __restrict__ d_t, const T * __restrict__ d_x, const T v, const T x1, const T x2, const T alpha, const T dt,
	const int tid, const int N) {

	if (tid == 0) 	{ 
		d_u2[0] = ((T)0.5) * (propagatingFunctionStationary(-v * d_t[1], x1, x2) - propagatingFunctionStationary(v * d_t[1], x1, x2)); 
		d_u[N + 1] = d_u2[0];
		return; }
	if (tid == N) { 
		d_u2[N] = ((T)0.5) * (propagatingFunctionStationary(x2 - x1 - v * d_t[1], x1, x2) - propagatingFunctionStationary(x2 - x1 + v * d_t[1], x1, x2)); 
		d_u[2 * N + 1] = d_u2[N];
		return;
	}
	d_u2[tid] = alpha * alpha * d_u1[tid + 1] / ((T)2) + ((T)1 - alpha * alpha) * d_u1[tid]
			+ alpha * alpha * d_u1[tid - 1] / ((T)2) - v * dt * ((T)0.5) * (
			propagatingFunctionStationaryDerivative(d_x[tid] - x1 - v * d_t[0], x1, x2) +
			propagatingFunctionStationaryDerivative(d_x[tid] - x1 + v * d_t[0], x1, x2));

	d_u[tid + N + 1] = d_u2[tid]; }

/**************************************/
/* UPDATE DEVICE FUNCTION - NO SHARED */
/**************************************/
template<class T>
__device__ void updateDeviceNoShared(T * __restrict__ d_u1, T * __restrict__ d_u2, T * __restrict__ d_u3, T * __restrict__ d_u, const T * __restrict__ d_t, const T * __restrict__ d_x, const T v, const T x1,
	const T x2, const T alpha, const int l, const int tid, const int N) {
    
    if (tid == 0) {
		d_u3[0] = ((T)0.5) * (propagatingFunctionStationary(-v * d_t[l + 1], x1, x2) - propagatingFunctionStationary(v * d_t[l + 1], x1, x2));
		d_u[(l + 1) * (N + 1)] = d_u3[0];
		return;
	}

	if (tid < N) {
		d_u3[tid] = alpha * alpha * d_u2[tid + 1] + ((T)2) * ((T)1 - alpha * alpha) * d_u2[tid] + alpha * alpha * d_u2[tid - 1] - d_u1[tid];
		d_u[(l + 1) * (N + 1) + tid] = d_u3[tid];
		return;
	}

	d_u3[N] = ((T)0.5) * (propagatingFunctionStationary(x2 - x1 - v * d_t[l + 1], x1, x2) - propagatingFunctionStationary(x2 - x1 + v * d_t[l + 1], x1, x2));
	d_u[(l + 1) * (N + 1) + N] = d_u3[N]; }


/***********************************/
/* UPDATE DEVICE FUNCTION - SHARED */
/***********************************/
template<class T>
__device__ void updateDeviceShared(T * __restrict__ d_u1, T * __restrict__ d_u2, T * __restrict__ d_u3, T * __restrict__ d_u, const T * __restrict__ d_t, const T * __restrict__ d_x, const T v, const T x1,
	const T x2, const T alpha, const int l, const int tid, const int N) {

	__shared__ T d_u2_temp[BLOCKSIZE + 2];

	d_u2_temp[threadIdx.x + 1] = d_u2[tid];

	if ((threadIdx.x == 0) && ((tid >= 1))) { d_u2_temp[0] = d_u2[tid - 1]; }										

	if ((threadIdx.x == 0) && ((tid + BLOCKSIZE) < (N + 1))) { d_u2_temp[BLOCKSIZE + 1] = d_u[tid + BLOCKSIZE]; }	

	__syncthreads();
	
	if (tid == 0) {
		d_u3[0] = ((T)0.5) * (propagatingFunctionStationary(-v * d_t[l + 1], x1, x2) - propagatingFunctionStationary(v * d_t[l + 1], x1, x2));
		d_u[(l + 1) * (N + 1)] = d_u3[0];
		return;
	}

	if (tid < N) {
		d_u3[tid] = alpha * alpha * d_u2_temp[threadIdx.x + 2] + ((T)2) * ((T)1 - alpha * alpha) * d_u2_temp[threadIdx.x + 1] + alpha * alpha * d_u2_temp[threadIdx.x] - d_u1[tid];
		d_u[(l + 1) * (N + 1) + tid] = d_u3[tid];
		return;
	}

	d_u3[N] = ((T)0.5) * (propagatingFunctionStationary(x2 - x1 - v * d_t[l + 1], x1, x2) - propagatingFunctionStationary(x2 - x1 + v * d_t[l + 1], x1, x2));
	d_u[(l + 1) * (N + 1) + N] = d_u3[N]; }
         
/***************************/
/* EXTERN GLOBAL FUNCTIONS */
/***************************/
extern "C" {

// --- Step0
__global__ void setStep0Kernel(float * __restrict__ d_u1, float * __restrict__ d_u, const float * __restrict__ d_t, const float * __restrict__ d_x, const float v, const float x1, const float x2, const int N) {

	const int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if (tid >= N + 1) return;

	setStep0Device(d_u1, d_u, d_t, d_x, v, x1, x2, tid); }

// --- Step1
__global__ void setStep1Kernel(float * __restrict__ d_u1, float * __restrict__ d_u2, float * __restrict__ d_u, const float * __restrict__ d_t, const float * __restrict__ d_x, const float v, const float x1, const float x2, const float alpha, const float dt,
	const int N) {

	const int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if (tid >= N + 1) return;

  setStep1Device(d_u1, d_u2, d_u, d_t, d_x, v, x1, x2, alpha, dt,	tid, N);
}

// --- Update kernel no shared 
__global__ void updateKernelNoShared(float * __restrict__ d_u1, float * __restrict__ d_u2, float * __restrict__ d_u3, float * __restrict__ d_u, const float * __restrict__ d_t, const float * __restrict__ d_x, const float v, const float x1,
	const float x2, const float alpha, const int l, const int N) {

	const int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if (tid >= N + 1) return;

  updateDeviceNoShared(d_u1, d_u2, d_u3, d_u, d_t, d_x, v, x1, x2, alpha, l, tid, N);
}

// --- Update kernel shared
__global__ void updateKernelShared(float * __restrict__ d_u1, float * __restrict__ d_u2, float * __restrict__ d_u3, float * __restrict__ d_u, const float * __restrict__ d_t, const float * __restrict__ d_x, const float v, const float x1,
	const float x2, const float alpha, const int l, const int N) {

	const int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if (tid >= N + 1) return;

  updateDeviceShared(d_u1, d_u2, d_u3, d_u, d_t, d_x, v, x1, x2, alpha, l, tid, N); }

}
""" % { "BBLOCKSIZE" : BLOCKSIZE }, no_extern_c = True)

Set references to the ```__global__``` functions.

In [6]:
setStep0                    = mod.get_function("setStep0Kernel")
setStep1                    = mod.get_function("setStep1Kernel")
updateNoShared              = mod.get_function("updateKernelNoShared")
updateShared                = mod.get_function("updateKernelShared")

Solution function. Rows span time, columns span space. We have ```M + 1``` rows: ```M``` iterations + ```1``` initial condition. We have ```N + 1``` columns: ```N``` discretization points + ```1``` boundary condition.

Leapfrog.

In [7]:
def leapfrogStationary(d_x, d_t, v, xmin, xmax, alpha):

  # --- Allocate device memory space for solution
  d_u  = cuda.mem_alloc((N + 1) * (M + 1) * 4)
  d_u1 = cuda.mem_alloc((N + 1)           * 4)
  d_u2 = cuda.mem_alloc((N + 1)           * 4)
  d_u3 = cuda.mem_alloc((N + 1)           * 4)
  # --- Set memory to zero
  cuda.memset_d32(d_u , 0x00, (N + 1) * (M + 1))
  cuda.memset_d32(d_u1, 0x00, (N + 1)          )
  cuda.memset_d32(d_u2, 0x00, (N + 1)          )
  cuda.memset_d32(d_u3, 0x00, (N + 1)          )

  blockDim  = (BLOCKSIZE, 1, 1)
  gridDim   = (int(iDivUp(N + 1, BLOCKSIZE)), 1, 1)

  # --- Step0
  setStep0(d_u1, d_u, d_t, d_x, np.float32(v), np.float32(xmin), np.float32(xmax), np.int32(N), block = blockDim, grid = gridDim)

  # --- Step1
  setStep1(d_u1, d_u2, d_u, d_t, d_x, np.float32(v), np.float32(xmin), np.float32(xmax), np.float32(alpha), np.float32(dt), np.int32(N), block = blockDim, grid = gridDim)

  for l in range(1, M - 1):

    updateShared(d_u1, d_u2, d_u3, d_u, d_t, d_x, np.float32(v), np.float32(xmin), np.float32(xmax), np.float32(alpha), np.int32(l), np.int32(N), block = blockDim, grid = gridDim)
    # updateNoShared(d_u1, d_u2, d_u3, d_u, d_t, d_x, np.float32(v), np.float32(xmin), np.float32(xmax), np.float32(alpha), np.int32(l), np.int32(N), block = blockDim, grid = gridDim)

    cuda.memcpy_dtod(d_u1, d_u2, (N + 1) * 4)
    cuda.memcpy_dtod(d_u2, d_u3, (N + 1) * 4)

  return d_u

Parameters.

In [8]:
xmin  = 0                                    # --- Left boundary of the simulation domain
xmax  = 2. * np.pi                           # --- Right boundary of the simulation domain
t_0   = 0.                                   # --- Initial time
t_f   = 15.                                  # --- Final time
M     = 200                                  # --- Number of time steps
N     = 100                                  # --- Number of space mesh points
v     = 0.5                                  # --- Wave speed

Space-time discretization.

In [9]:
x, dx = np.linspace(xmin, xmax, N + 1, retstep = True, dtype = np.float32)

t, dt = np.linspace(t_0, t_f, M + 1, retstep = True, dtype = np.float32)
            
alpha = v * dt / dx

Moving discretization to the device.

In [10]:
d_x = cuda.mem_alloc(x.nbytes)
d_t = cuda.mem_alloc(t.nbytes)

cuda.memcpy_htod(d_x, x)
cuda.memcpy_htod(d_t, t)

Compute solution.

In [11]:
d_u = leapfrogStationary(d_x, d_t, v, xmin, xmax, alpha)

Copy the solution from device to host.

In [12]:
u = np.zeros((M + 1, N + 1), dtype = np.float32)
cuda.memcpy_dtoh(u, d_u)

Propagating distribution and its derivative. They serve as reference and as initial condition.

In [13]:
def propagatingFunctionStationary(x, xmin, xmax):
    return np.cos((2. * np.pi / ((2. / 3.) * (xmax - xmin))) * x)

def propagatingFunctionStationaryDerivative(x, xmin, xmax):
  return -(2. * np.pi / ((2. / 3.) * (xmax - xmin))) * np.sin((2. * np.pi / ((2. / 3.) * (xmax - xmin))) * x)

Define reference solution.

In [14]:
X, T = np.meshgrid(x, t)           

uFW  =  0.5 * propagatingFunctionStationary(X - xmin - v * T, xmin, xmax);
uBW  = -0.5 * propagatingFunctionStationary(X - xmin + v * T, xmin, xmax);

uRef = 0.5 * (propagatingFunctionStationary(X - xmin - v * T, xmin, xmax) - propagatingFunctionStationary(X - xmin + v * T, xmin, xmax));  

Animation.

In [15]:
fig, ax = plt.subplots()
plt.close()

ax.set_xlim(( xmin, xmax))
ax.set_ylim(( -1.1, 1.1))

line1, = ax.plot([], [], lw = 2, color = "r")
line2, = ax.plot([], [], lw = 2, color = "g")
line3, = ax.plot([], [], lw = 2, color = "c")
line4, = ax.plot([], [], lw = 2, color = "b", marker = "o")

def animate(i):
    global x
    global u, uRef, uFW, uBW
    yRef = uRef[i]
    yFW  = uFW[i]
    yBW  = uBW[i]
    y    = u[i]
    line1.set_data(x, yRef)
    line2.set_data(x, yFW)
    line3.set_data(x, yBW)
    line4.set_data(x, y)
    return (line1, line2, line3, line4)

anim = animation.FuncAnimation(fig, animate, frames = N + 1, interval = 20)
anim.save('leapfrog.mp4', fps = 30, extra_args=['-vcodec', 'libx264'])
rc('animation', html = 'jshtml') 
anim